In [1]:
%pip install easyocr
%pip install transformers==4.41.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
import sys
import cv2
import torch
import numpy as np
from collections import Counter
from torchvision import transforms
from PIL import Image

from Tex2loc import Tex2loc
from GeoGessr import GeoGuessCountryClassifier
from tekstoinator import extract_text_from_video

def load_classifier_model(model_path, device, num_classes):
    model = GeoGuessCountryClassifier(num_classes=num_classes)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

def preprocess_frame(frame):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    return preprocess(frame)

def predict_country_on_video(video_path, model, device, label_encoder):
    cap = cv2.VideoCapture(video_path)
    predictions = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1

        img_tensor = preprocess_frame(frame).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = model(img_tensor)
            _, pred = torch.max(outputs, 1)
            predictions.append(pred.item())

    cap.release()

    most_common_idx = Counter(predictions).most_common(1)[0][0]
    most_common_country = label_encoder.classes_[most_common_idx]
    return most_common_country


In [ ]:
video_path = "input/video.mp4"

if not os.path.isfile(video_path):
    print(f"Video file '{video_path}' nie istnieje!")
    sys.exit(1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Używany device: {device}")

print("1. Wykonywanie OCR na video...")
extracted_text_list = extract_text_from_video(video_path, frame_interval=42)
extracted_text = " ".join(extracted_text_list)
print(f"Znaleziony tekst (fragment): {extracted_text[:500]}...")



Używany device: cuda
1. Wykonywanie OCR na video...


In [5]:
print(extracted_text)
print("2. Analiza lokalizacji z tekstu...")
tex2loc = Tex2loc(device=device.type)

BURGERA [RABEN CZASAA ABEN CZAS RABEN WYINLE
2. Analiza lokalizacji z tekstu...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [6]:

location_info = tex2loc.get_location_info(extracted_text)
print(f"Lokalizacja z tekstu: {location_info}")



/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Lokalizacja z tekstu: {'city': 'Unknown', 'country': 'Unknown', 'continent': 'Europe', 'latitude': 12.4989994, 'longitude': 124.6746741}


In [8]:
print("3. Ładowanie klasyfikatora obrazów...")

import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("dataset/country_dataset.csv", header=None, names=['country', 'lat', 'lon', 'local_path'])

counts = df['country'].value_counts()
valid_countries = counts[counts >= 2].index
df = df[df['country'].isin(valid_countries)]

countries = sorted(df['country'].unique())

le = LabelEncoder()
le.fit(countries)

model_path = "best_model.pt"
model = load_classifier_model(model_path, device, num_classes=158)
print("Model załadowany.")

print("4. Predykcje na klatkach video...")
most_common_country = predict_country_on_video(video_path, model, device, le)
print(f"Najczęściej przewidywany kraj na video: {most_common_country}")

print("\n--- Podsumowanie ---")
print(f"Tekst OCR: {extracted_text[:500]}...")
print(f"Lokalizacja tekstowa: Miasto: {location_info['city']}, Kraj: {location_info['country']}, Kontynent: {location_info['continent']}")
print(f"Predykcja modelu na klatkach: {most_common_country}")

3. Ładowanie klasyfikatora obrazów...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model załadowany.
4. Predykcje na klatkach video...
Najczęściej przewidywany kraj na video: Chad

--- Podsumowanie ---
Tekst OCR: BURGERA [RABEN CZASAA ABEN CZAS RABEN WYINLE...
Lokalizacja tekstowa: Miasto: Unknown, Kraj: Unknown, Kontynent: Europe
Predykcja modelu na klatkach: Chad
